In [10]:
import pulp as p
import numpy as np

def init_lp():
	# Init solver and variables
	Lp_prob = p.LpProblem('StochasticCostMinimization', p.LpMinimize)

	# First enterprise vs resource 1 transportation
	x11 = p.LpVariable("x11", lowBound=0)  # x11 >= 0
	x12 = p.LpVariable("x12", lowBound=0)  # x12 >= 0
	x13 = p.LpVariable("x13", lowBound=0)  # x13 >= 0
	x14 = p.LpVariable("x14", lowBound=0)  # x14 >= 0

	# Second enterprise vs resource 1 transportation
	x21 = p.LpVariable("x21", lowBound=0)  # x21 >= 0
	x22 = p.LpVariable("x22", lowBound=0)  # x22 >= 0
	x23 = p.LpVariable("x23", lowBound=0)  # x23 >= 0
	x24 = p.LpVariable("x24", lowBound=0)  # x24 >= 0

	# First enterprise vs resource 2 transportation
	y11 = p.LpVariable("y11", lowBound=0)  # y11 >= 0
	y12 = p.LpVariable("y12", lowBound=0)  # y12 >= 0
	y13 = p.LpVariable("y13", lowBound=0)  # y13 >= 0

	# Second enterprise vs resource 2 transportation
	y21 = p.LpVariable("y21", lowBound=0)  # y21 >= 0
	y22 = p.LpVariable("y22", lowBound=0)  # y22 >= 0
	y23 = p.LpVariable("y23", lowBound=0)  # y23 >= 0

	# First enterprise vs warehouses
	q11 = p.LpVariable("q11", lowBound=0, cat='Integer')  # q11 >= 0, integer
	q12 = p.LpVariable("q12", lowBound=0, cat='Integer')  # q12 >= 0, integer
	q13 = p.LpVariable("q13", lowBound=0, cat='Integer')  # q13 >= 0, integer
	q14 = p.LpVariable("q14", lowBound=0, cat='Integer')  # q14 >= 0, integer

	# Second enterprise vs warehouses
	q21 = p.LpVariable("q21", lowBound=0, cat='Integer')  # q21 >= 0, integer
	q22 = p.LpVariable("q22", lowBound=0, cat='Integer')  # q22 >= 0, integer
	q23 = p.LpVariable("q23", lowBound=0, cat='Integer')  # q23 >= 0, integer
	q24 = p.LpVariable("q24", lowBound=0, cat='Integer')  # q24 >= 0, integer

	# Production
	z1 = p.LpVariable("z1", lowBound=0, cat='Integer')  # z1 >= 0, integer
	z2 = p.LpVariable("z2", lowBound=0, cat='Integer')  # z2 >= 0, integer

	# Common constraints for pessimista/optimista models
	Lp_prob += x11 + x12 + x13 + x14 == 4 * (y11 + y12 + y13)
	Lp_prob += x21 + x22 + x23 + x24 == 5 * (y21 + y22 + y23)
	Lp_prob += z1 == 10 * (x11 + x12 + x13 + x14)
	Lp_prob += z2 == 8 * (x21 + x22 + x23 + x24)

	Lp_prob += q11 + q21 == 1000
	Lp_prob += q12 + q22 == 1200
	Lp_prob += q13 + q23 == 1500
	Lp_prob += q14 + q24 == 2000

	Lp_prob += q11 + q12 + q13 + q14 == z1
	Lp_prob += q21 + q22 + q23 + q24 == z2
	Lp_prob += z1 + z2 == 5700

	return x11, x12, x13, x14, x21, x22, x23, x24, \
		   y11, y12, y13, y21, y22, y23, \
		   q11, q12, q13, q14, \
		   q21, q22, q23, q24, \
		   z1, z2, Lp_prob

def variance_to_fuzzy_coefficient(sigma):
	return sigma * np.sqrt(2 * np.log(2.26 * sigma))

In [11]:
x11, x12, x13, x14, x21, x22, x23, x24, y11, y12, y13, y21, y22, y23, q11, q12, q13, q14, q21, q22, q23, q24, z1, z2, Lp_pessimista  = init_lp()
# Pessimista problem
Lp_pessimista += 	    220 * (x11 + x21) + 240 * (x12 + x22) + 140 * (x13 + x23) + 200 * (x14 + x24) + \
						50 * (y11 + y21) + 60 * (y12 + y22) + 70 * (y13 + y23) + \
                        20 * x11 + 70 * x12 + 50 * x13 + 40 * x14 + \
                        60 * x21 + 40 * x22 + 50 * x23 + 20 * x24 + \
                        50 * y11 + 20 * y12 + 40 * y13 + \
                        20 * y21 + 40 * y22 + 20 * y23 + \
                        5 * q11 + 4 * q12 + 5 * q13 + 10 * q14 + \
						variance_to_fuzzy_coefficient(5) * q11 + \
						variance_to_fuzzy_coefficient(4) * q12 + \
						variance_to_fuzzy_coefficient(5) * q13 + \
						variance_to_fuzzy_coefficient(10) * q14 + \
                        5 * q21 + 3 * q22 + 6 * q23 + 10 * q24 + \
						variance_to_fuzzy_coefficient(5) * q21 + \
						variance_to_fuzzy_coefficient(4) * q22 + \
						variance_to_fuzzy_coefficient(5) * q23 + \
						variance_to_fuzzy_coefficient(10) * q24 + \
		   				577800 + 96 * z1 + 804600 + 48 * z2

Lp_pessimista.solve()

# Printing the final solution
print("X by 1st enterprise: {}".format((p.value(x11), p.value(x12), p.value(x13), p.value(x14))))
print("X by 2nd enterprise: {}".format((p.value(x21), p.value(x22), p.value(x23), p.value(x24))))
print("Y by 1st enterprise: {}".format((p.value(y11), p.value(y12), p.value(y13))))
print("Y by 2nd enterprise: {}".format((p.value(y21), p.value(y22), p.value(y23))))
print("Q by 1st enterprise: {}".format((p.value(q11), p.value(q12), p.value(q13), p.value(q14))))
print("Q by 2nd enterprise: {}".format((p.value(q21), p.value(q22), p.value(q23), p.value(q24))))
print("Z: {}".format((p.value(z1), p.value(z2))))

result = p.value(Lp_pessimista.objective)
print("Pessimista objective function value: {}".format(result))

X by 1st enterprise: (0.0, 0.0, 0.0, 0.0)
X by 2nd enterprise: (0.0, 0.0, 712.5, 0.0)
Y by 1st enterprise: (0.0, 0.0, 0.0)
Y by 2nd enterprise: (142.5, 0.0, 0.0)
Q by 1st enterprise: (0.0, 0.0, 0.0, 0.0)
Q by 2nd enterprise: (1000.0, 1200.0, 1500.0, 2000.0)
Z: (0.0, 5700.0)
Pessimista objective function value: 1926493.2030251424


In [12]:
x11, x12, x13, x14, x21, x22, x23, x24, y11, y12, y13, y21, y22, y23, q11, q12, q13, q14, q21, q22, q23, q24, z1, z2, Lp_optimista  = init_lp()
# Optimista problem
Lp_optimista +=        220 * (x11 + x21) + 240 * (x12 + x22) + 140 * (x13 + x23) + 200 * (x14 + x24) + \
						50 * (y11 + y21) + 60 * (y12 + y22) + 70 * (y13 + y23) + \
                        20 * x11 + 70 * x12 + 50 * x13 + 40 * x14 + \
                        60 * x21 + 40 * x22 + 50 * x23 + 20 * x24 + \
                        50 * y11 + 20 * y12 + 40 * y13 + \
                        20 * y21 + 40 * y22 + 20 * y23 + \
                        5 * q11 + 4 * q12 + 5 * q13 + 10 * q14 + \
						-variance_to_fuzzy_coefficient(5) * q11 + \
						-variance_to_fuzzy_coefficient(4) * q12 + \
						-variance_to_fuzzy_coefficient(5) * q13 + \
						-variance_to_fuzzy_coefficient(10) * q14 + \
                        5 * q21 + 3 * q22 + 6 * q23 + 10 * q24 + \
						- variance_to_fuzzy_coefficient(5) * q21 + \
						- variance_to_fuzzy_coefficient(4) * q22 + \
						- variance_to_fuzzy_coefficient(5) * q23 + \
						- variance_to_fuzzy_coefficient(10) * q24 + \
		   				577800 + 96 * z1 + 804600 + 48 * z2

Lp_optimista.solve()

# Printing the final solution
print("X by 1st enterprise: {}".format((p.value(x11), p.value(x12), p.value(x13), p.value(x14))))
print("X by 2nd enterprise: {}".format((p.value(x21), p.value(x22), p.value(x23), p.value(x24))))
print("Y by 1st enterprise: {}".format((p.value(y11), p.value(y12), p.value(y13))))
print("Y by 2nd enterprise: {}".format((p.value(y21), p.value(y22), p.value(y23))))
print("Q by 1st enterprise: {}".format((p.value(q11), p.value(q12), p.value(q13), p.value(q14))))
print("Q by 2nd enterprise: {}".format((p.value(q21), p.value(q22), p.value(q23), p.value(q24))))
print("Z: {}".format((p.value(z1), p.value(z2))))

result = p.value(Lp_optimista.objective)
print("Optimista objective function value: {}".format(result))

X by 1st enterprise: (0.0, 0.0, 0.0, 0.0)
X by 2nd enterprise: (0.0, 0.0, 712.5, 0.0)
Y by 1st enterprise: (0.0, 0.0, 0.0)
Y by 2nd enterprise: (142.5, 0.0, 0.0)
Q by 1st enterprise: (0.0, 0.0, 0.0, 0.0)
Q by 2nd enterprise: (1000.0, 1200.0, 1500.0, 2000.0)
Z: (0.0, 5700.0)
Optimista objective function value: 1751406.7969748576
